Imperative Programming
=================

Many problems don't fit cleanly into `ndarray` or `DataFrame` abstractions.  How can we use dask to parallelize more custom workloads?

We can always fall back to creating dictionaries manually:

    dsk = {'load-1': (load, filename1), 'clean-1': (clean, 'load-1'), ...,
           'load-2': (load, filename2), 'clean-2': (clean, 'load-2'), ...,
           ...}
    
Manual dictionary creation though can be tedious, is prone to programmer error, and feels foreign to many developers. 

The dask `do` function helps you to construct custom dask graphs using more typical coding styles than the explicit construction of a dictionary.

### Custom graphs with `do`

The `do` function delays a function evaluation, producing a lazily evaluated result.  One wraps a function with a `do` call

*  Before:  

        result = f(a, b, c=10)
*  After:  

        result = do(f)(a, b, c=10)
        
The result of a call to `do(function)` is a lazy `Value` object that we can use in future `do` calls or eventually call `.compute()`

    >>> result.compute()

### A Familiar Example

To explore this abstraction we revisit our examples from the [Foundations Notebook](02-Foundations.ipynb)

In [ ]:
def inc(x):
    return x

def add(x, y):
    return x + y

a = 1
b = inc(a)

x = 10
y = inc(x)

z = add(b, y)
z

Originally we parallelized this by constructing a dask graph explicitly

In [ ]:
dsk = {'a': 1, 
       'b': (inc, 'a'),
       
       'x': 10,
       'y': (inc, 'x'),
       
       'z': (add, 'b', 'y')}

Now we can also use the `do` function to construct the dask graph with more traditional programming.

In [ ]:
from dask import do

a = 1
b = do(inc)(a)

x = 10
y = do(inc)(x)

z = do(add)(b, y)
z

In [ ]:
z.compute()

These value objects build up the dask graph as they go.  These graphs are less interpretable but fine for normal execution.

In [ ]:
z.dask

Exercise
---------

Consider our first exercise reading three CSV files with `pd.read_csv` and then measuring their total length.  

In [ ]:
import pandas as pd
import os
filenames = [os.path.join('data', 'accounts.%d.csv' % i) for i in [0, 1, 2]]
filenames

In [ ]:
%%time

a = pd.read_csv(filenames[0])
b = pd.read_csv(filenames[1])
c = pd.read_csv(filenames[2])

na = len(a)
nb = len(b)
nc = len(c)

total = sum([na, nb, nc])
total

In the first notebook we constructed a dask graph from this computation and then executed it in parallel using multiple processes to get a speedup

In [ ]:
# %load solutions/Foundations-01.py
dsk = {'a': (pd.read_csv, filenames[0]),
       'b': (pd.read_csv, filenames[1]),
       'c': (pd.read_csv, filenames[2]),
       'na': (len, 'a'),
       'nb': (len, 'b'),
       'nc': (len, 'c'),
       'total': (sum, ['na', 'nb', 'nc'])}

In [ ]:
from dask.multiprocessing import get
%time  get(dsk, 'total')

Your task is to recreate this graph again using the `do` function on the original Python code.

In [ ]:
a = do(pd.read_csv)(filenames[0])
...

total = ...

%time total.compute(get=get) # use multiprocessing get function in call to compute

In [ ]:
# Solution
%load solutions/Imperative-01.py

Exercise
---------

Below is a function that approximates Pi using a [Monte Carlo method](https://en.wikipedia.org/wiki/Monte_Carlo_method). It works by generating random points in a 1 x 1 square, and then counting those that are inside a quarter circle of radius one (as seen in [this gif](https://en.wikipedia.org/wiki/Monte_Carlo_method#/media/File:Pi_30K.gif)). Since the area of the full circle is Pi, then this can be estimated by 4 x points_in_circle/total_points.

In [ ]:
from __future__ import division
from random import random

def estimate_pi(nsamples):
    total = 0
    for i in range(nsamples):
        x = random()
        y = random()
        if x*x + y*y <= 1:
            total += 1
    return 4.*total/nsamples

estimate_pi(3000)

Your task is to parallelize this computation using the `do` function.


Think about what can be done in parallel here. The computation can be broken down into:

1. Generate a bunch of random points
2. For each point, determine if it's in the circle. If so, increment total counter.
3. Return 4 * total/nsamples
    
This is basically a map operation (for each point do something), followed by a reduction (final aggregation arithmetic), so it can be done in an extremely parallel fashion, with the calculation for each point being a task. Below is a skeleton of a parallel version of the code, with the computation for each point missing.

In [ ]:
@do
def point_chunk():
    """Generates a random x, y point, returns 1 if in circle, else returns 0."""
    ...

def parallel_estimate_pi(nsamples):
    points = []
    for i in range(nsamples):
        points.append(point_chunk())
    total = do(sum)(points)
    result = 4.*total/nsamples
    return result.compute(get=get)

parallel_estimate_pi(3000)

In [ ]:
%time estimate_pi(3000)

In [ ]:
%time parallel_estimate_pi(3000)

In [ ]:
# Solution
%load solutions/Imperative-02.py

How fast did that run? Was it faster or slower than the serial version? 

Even though there was a high amount of parallelism here, it ran significantly slower. This is because of the overhead of the dask schedulers. Dask is really good at "large grain parallelism", but when the task size gets to be too small then the scheduler overhead becomes a problem. 

This can be fixed by partioning the computation into larger blocks. Below is another take on the same computation. This time we partition the total number of samples into `npartitions` using the provided `partition` function. We then apply `total_chunk` to each partition, sum the results, then apply the arithmetic to calculate Pi. In this way, each chunk is more computation heavy, so the overhead from the schedulers is smaller. Finish up the implementation by writing `total_chunk`.

In [ ]:
def partition(num, npartitions):
    """Partition `num` into `npartitions` partitions.
    
    >>> partition(10, 3)
    [3, 3, 4]
    """
    parts = [num//npartitions] * npartitions
    parts[-1] += num%npartitions
    return parts

In [ ]:
@do
def total_chunk(nsamples):
    """Generates `nsamples` random x, y points, returns number of 
    points that were in the circle.
    """
    ...

def parallel_estimate_pi(nsamples, npartitions):
    totals = []
    for n in partition(nsamples, npartitions):
        totals.append(total_chunk(n))
    result = 4.*do(sum)(totals)/nsamples
    return result.compute(get=get)

parallel_estimate_pi(3000, 8)

In [ ]:
%time estimate_pi(10000000)

In [ ]:
%time parallel_estimate_pi(10000000, 8)

In [ ]:
# Solution
%load solutions/Imperative-03.py

This is significantly faster than our original parallel implementation, and also runs faster than the serial version. The difference is even more apparent on larger sample sizes:

In [ ]:
%time estimate_pi(100000000)

In [ ]:
%time parallel_estimate_pi(100000000, 8)